In [1]:
import pandas as pd
from tqdm.autonotebook import tqdm
import re

/tmp/ipykernel_195630/3235371013.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
df = pd.read_csv("paragrapgs_with_topics.csv.zip", index_col=0)
df = df.dropna()
df = df[df["cleaned_text"].apply(lambda x: len(x.split()) > 1)]
df = df.reset_index(drop=True)
df.head()

/tmp/ipykernel_195630/2910252601.py:1: DtypeWarning: Columns (2,17,27,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("paragrapgs_with_topics.csv.zip", index_col=0)


,company,year,sector,report_type,paragraph,original_text,cleaned_text,Экология,Климат,Энергия,...,Обучение и развитие,Оплата труда,Отношения потребителями,Отношения с потребителями,Отношения с работниками,Охрана здоровья,Профсоюзы и коллективные договоры,Трудовые отношения,max_scoreG,max_topicG
0,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,9.0,"Виктор ФИЛИПпОВИЧ Рашников, председатель совет...",виктор филиппович рашник председатель совет ди...,0.130189,0.092450,0.0,...,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.051988,Инвестиции и капитальные вложения
1,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,11.0,Минувший 2014 год предъявил всем отраслям росс...,минувший год предъявить весь отрасль российски...,0.000000,0.092450,0.0,...,0.0,0.08737,0.000000,0.145095,0.000000,0.0,0.0,0.000000,0.145095,Отношения с потребителями
2,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,12.0,Для Магнитогорского металлургического комбинат...,для магнитогорский металлургический комбинат г...,0.000000,0.130744,0.0,...,0.0,0.00000,0.141421,0.102598,0.109109,0.0,0.0,0.056523,0.141421,Отношения потребителями
3,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,13.0,В 2014 году ММК укрепил свои позиции ведущего ...,в год ммк укрепить свой позиция ведущий постав...,0.130189,0.092450,0.0,...,0.0,0.00000,0.000000,0.102598,0.000000,0.0,0.0,0.000000,0.157135,Налоги
4,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,14.0,Наша компания входит в число российских предпр...,наш компания входить число российский предприя...,0.130189,0.092450,0.0,...,0.0,0.00000,0.000000,0.102598,0.000000,0.0,0.0,0.079936,0.102598,Отношения с потребителями


In [3]:
#df.join(topics).to_csv("paragrapgs_with_topics.csv.zip")

In [4]:
df["max_topic"].value_counts()

Экология           70346
Климат             51830
Энергия            19316
Отходы             14381
Вода               10253
Биоразнообразие     2084
Воздух              1839
Name: max_topic, dtype: int64

In [5]:
df["max_topicG"].value_counts()

Безопасность продукта                22149
Отношения с потребителями            21898
Отношения с работниками              20946
Налоги                               20193
Инвестиции и капитальные вложения    18596
Трудовые отношения                   17026
Безопасность и охрана труда          11619
Обучение и развитие                   9530
Отношения  потребителями              8550
Благотворительность                   8171
Профсоюзы и коллективные договоры     6310
Оплата труда                          2619
Охрана здоровья                       2442
Name: max_topicG, dtype: int64

In [6]:
df["max_topicS"].value_counts()

Управление рисками                    34452
Эффективность и производительность    33457
Антикоррупция                         27487
Дивиденды и акционеры                 20185
Предотвращение коррупции              19303
Лидерство                             16842
Инновации                             11849
Отчетность и прозрачность              6474
Name: max_topicS, dtype: int64

In [8]:
grouped = df.groupby(["company", "year", "max_topic"])["cleaned_text"].agg(' '.join)
grouped.head()

company      year        max_topic      
Abbott       2013, 2012  Биоразнообразие    недостаточность питание повышенный потребность...
                         Климат             программа проводиться июнь декабрь год направи...
                         Отходы             о компания это один крупный многопрофильный ко...
                         Экология           молекулярный диагностика детский лечебный пита...
Alcoa, Inc.  2004        Воздух                        окисел азот в тысяча метрический тонна
Name: cleaned_text, dtype: object

In [9]:
from collections import defaultdict

sheets_dict = {}
base_dict = {}
re_dict = {}
len_topics = {}

excel = pd.ExcelFile("./esg_topics.xlsx")
for sheet_name in excel.sheet_names:
    sheet_df = pd.read_excel(excel, sheet_name=sheet_name, header=None)
    sheets_dict[sheet_name] = {row[0]: row[1] for _, row in sheet_df.iterrows()}
    base_dict[sheet_name] = {row[0]: 0 for _, row in sheet_df.iterrows()}
    re_dict[sheet_name] = re.compile("(" + "(?!\w)|".join(sheet_df[0].tolist()) + "(?!\w))")
    len_topics[sheet_name] = defaultdict(int)
    for val in sheets_dict[sheet_name].values():
        len_topics[sheet_name][val] += 1

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [7]:
excel = pd.ExcelFile("./Рейтинг ESG - Отчеты компаний.xlsx")
sheets = {}
for sheet_name in ["E топики", "G топики", "S топики"]:
    sheets[sheet_name] = pd.read_excel(excel, sheet_name=sheet_name)

In [12]:
words = {}
words["G"] = sheets["G топики"].groupby("УНИВЕРСАЛЬНЫЕ ТОПИКИ")["topic_words"].apply(", ".join)
words["S"] = sheets["S топики"].groupby("УНИВЕРСАЛЬНЫЕ ТОПИКИ")["topic_words"].apply(", ".join)
words["G"] = words["G"].str.split(", ").apply(set).apply(list)
words["S"] = words["S"].str.split(", ").apply(set).apply(list)

In [15]:
from collections import defaultdict

sheets_dict = {}
base_dict = {}
re_dict = {}
len_topics = {}

for topic, word in words["G"].items():
    sheets_dict[topic] = {row: 1 for row in word}
    base_dict[topic] = {row: 0 for row in word}
    re_dict[topic] = re.compile("(" + "(?!\w)|".join(word) + "(?!\w))")
    len_topics[topic] = defaultdict(int)
    for val in sheets_dict[topic].values():
        len_topics[topic][val] += 1

In [16]:
def get_score(words: dict[str, int], topic: str) -> int:
    count_words = {key: 0 for key in len_topics[topic]}
    for word in words:
        count_words[sheets_dict[topic][word]] += 1
    base_percent = count_words[1] / len_topics[topic][1]
    advanced_percent =  count_words[2] / len_topics[topic][2] if count_words.get(2, False) else 0
    if base_percent < 0.20:
        return 1
    elif base_percent < 0.40:
        return 2
    elif base_percent < 0.60:
        return 3
    elif base_percent < 0.80:
        return 4
    elif base_percent <= 1 and advanced_percent == 0:
        return 5
    elif base_percent <= 1 and advanced_percent < 0.40:
        return 6
    elif base_percent <= 1 and advanced_percent < 0.80:
        return 7
    elif base_percent <= 1 and advanced_percent <= 1:
        return 7
    else:
        return 0

In [17]:
grouped = df.groupby(["company", "year", "max_topicG"])["cleaned_text"].agg(' '.join)
grouped.head()

company       year  max_topicG                       
Alcoa, Inc.   2004  Отношения с потребителями                  белый калитва восток московский офис самара
                    Отношения с работниками              природосберегающий использоваться кроме упаков...
                    Профсоюзы и коллективные договоры    показатель экологический безопасность для комп...
BELUGA GROUP  2020  Безопасность и охрана труда          пао белуга группа общий информация по группа с...
                    Безопасность продукта                пао белуга группа общий информация по группа т...
Name: cleaned_text, dtype: object

In [ ]:
import copy


scores = []
percents = []
for (company, year, topic), text in tqdm(grouped.items(), total=grouped.shape[0]):
    paragraph_dict = copy.deepcopy(base_dict)[topic]
    words = re_dict[topic].findall(text)
    for word in words:
        paragraph_dict[word] = 1
    scores.append(get_score(paragraph_dict, topic))

  0%|          | 0/6971 [00:00<?, ?it/s]

In [ ]:
grouped = pd.DataFrame(grouped) 
grouped["score"] = scores
grouped.head()

In [ ]:
grouped.to_csv("grouped_paragraphsG.csv.zip")

In [14]:
grouped = pd.read_csv("grouped_paragraphs.csv.zip")

In [15]:
grouped.head()

,company,year,max_topic,cleaned_text,score
0,Abbott,"2013, 2012",Биоразнообразие,недостаточность питание повышенный потребность...,5
1,Abbott,"2013, 2012",Климат,программа проводиться июнь декабрь год направи...,7
2,Abbott,"2013, 2012",Отходы,о компания это один крупный многопрофильный ко...,7
3,Abbott,"2013, 2012",Экология,молекулярный диагностика детский лечебный пита...,7
4,"Alcoa, Inc.",2004,Воздух,окисел азот в тысяча метрический тонна,5
